In [ ]:
SELECT MONTH(START_DATE) as MONTH, CAR_ID, COUNT(CAR_ID) as RECORDS
FROM CAR_RENTAL_COMPANY_RENTAL_HISTORY
WHERE YEAR(START_DATE) = 2022 and MONTH(START_DATE) between 8 and 10
and CAR_ID IN (
    SELECT CAR_ID
    FROM CAR_RENTAL_COMPANY_RENTAL_HISTORY 
    WHERE YEAR(START_DATE) = 2022 and MONTH(START_DATE) between 8 and 10
    GROUP BY CAR_ID 
    HAVING count(*) >= 5
)
GROUP BY MONTH(START_DATE), CAR_ID
ORDER BY MONTH ASC, CAR_ID DESC

In [ ]:
SELECT CAR_ID,
    CASE 
        WHEN EXISTS (# CAR_Id 중에 하나가 조건에 만족 한다면
            SELECT 1
            FROM CAR_RENTAL_COMPANY_RENTAL_HISTORY as c2
            WHERE '2022-10-16' between START_DATE and END_DATE and c1.CAR_ID = c2.CAR_ID # 이조건이 있으면 외부 쿼리에서 CAR_ID가 1001일때 내부 쿼리도 그 값에 대한 것만 조건을 검사하기 때문에 나누어서  검사함 
        )
        THEN '대여중'
        ELSE '대여 가능'
    END AS AVAILABILITY
FROM CAR_RENTAL_COMPANY_RENTAL_HISTORY as c1
GROUP BY CAR_ID
ORDER BY CAR_ID DESC

다음의 쿼리는 내부 쿼리와 외부 쿼리로 나눌 수 있다.
외부 쿼리는 AVAILABILITY 컬럼에 대해 '대여중' 또는 '대여가능' 중 하나의 값을 반환하게 된다.
이 때 내부 쿼리는 외부 쿼리의 CAR_ID를 참조하며, 이를 상관 서브쿼리 (correlated subquery) 라고 한다.
만약 상관 서브쿼리가 아니었다면, 서브쿼리가 하나라도 참이면 항상 '대여중'만 반환됐을 것이다.
하지만 현재는 내부 쿼리가 외부 쿼리의 CAR_ID 값을 받아와서,
외부에서 보고 있는 차량(CAR_ID = c1)에 대해 내부 쿼리에서 같은 차량(CAR_ID = c1)의 대여 여부를 개별적으로 판단하게 되는 것이다.

In [ ]:
# 외부쿼리
SELECT CAR_ID,
    CASE 
        WHEN EXISTS ()
        THEN '대여중'
        ELSE '대여 가능'
    END AS AVAILABILITY
FROM CAR_RENTAL_COMPANY_RENTAL_HISTORY as c1
GROUP BY CAR_ID
ORDER BY CAR_ID DESC

# 내부쿼리
SELECT 1
FROM CAR_RENTAL_COMPANY_RENTAL_HISTORY as c2
WHERE '2022-10-16' between START_DATE and END_DATE and c1.CAR_ID = c2.CAR_ID
        

---

In [ ]:
SELECT FOOD_TYPE, REST_ID, REST_NAME, FAVORITES
FROM REST_INFO as r1
WHERE FAVORITES = (
    SELECT MAX(FAVORITES)
    FROM REST_INFO as r2
    WHERE r1.FOOD_TYPE = r2.FOOD_TYPE
    GROUP BY FOOD_TYPE
)
ORDER BY FOOD_TYPE DESC

In [ ]:
SELECT r1.FOOD_TYPE, r1.REST_ID, r1.REST_NAME, r1.FAVORITES
FROM REST_INFO as r1
JOIN (
    SELECT FOOD_TYPE, MAX(FAVORITES) as MAX_FAV
    FROM REST_INFO
    GROUP BY FOOD_TYPE
) as r2
ON r1.FOOD_TYPE = r2.FOOD_TYPE and r1.FAVORITES = r2.MAX_FAV
ORDER BY r1.FOOD_TYPE DESC

In [ ]:
SELECT MCDP_CD as 진료과코드, COUNT(*) as 5월예약건수
FROM APPOINTMENT
WHERE YEAR(APNT_YMD) = 2022 and MONTH(APNT_YMD) = 5
GROUP BY MCDP_CD
ORDER BY 5월예약건수, MCDP_CD

In [ ]:
SELECT INGREDIENT_TYPE, SUM(TOTAL_ORDER) as TOTAL_ORDER
FROM FIRST_HALF as f
JOIN ICECREAM_INFO as I
ON f.FLAVOR = I.FLAVOR
GROUP BY INGREDIENT_TYPE
ORDER BY TOTAL_ORDER

In [ ]:
SELECT CAR_TYPE, COUNT(*) as CARS
FROM CAR_RENTAL_COMPANY_CAR as c1
WHERE OPTIONS LIKE '%가죽시트%'
    OR OPTIONS LIKE '%통풍시트%'
    OR OPTIONS LIKE '%열선시트%'
GROUP BY CAR_TYPE 
ORDER BY CAR_TYPE ASC

In [ ]:
SELECT CATEGORY, SUM(SALES) AS TOTAL_SALES
FROM BOOK as b
JOIN BOOK_SALES as bs
ON b.BOOK_ID = bs.BOOK_ID
WHERE bs.SALES_DATE >= '2022-01-01' and bs.SALES_DATE < '2022-02-01'
GROUP BY CATEGORY
ORDER BY CATEGORY

In [ ]:
SELECT USER_ID, NICKNAME, SUM(PRICE) as TOTAL_SALES
FROM USED_GOODS_BOARD as ugb
JOIN USED_GOODS_USER as ugu
ON ugb.WRITER_ID = ugu.USER_ID
WHERE STATUS = 'DONE'
GROUP BY WRITER_ID
HAVING TOTAL_SALES >= 700000
ORDER BY TOTAL_SALES

ON 말고 USING을 사용하면 중복 컬럼이 사라짐 그래서 테이블 명을 굳이 적어주지 않아도 중복 컬럼 값이 알아서 선택 됨

In [ ]:
SELECT (PRICE DIV 10000)*10000 as PRICE_GROUP, COUNT(*) AS PRODUCTS
FROM PRODUCT
GROUP BY PRICE_GROUP
ORDER BY PRICE_GROUP

### ✅ SQL 수치형 연산자 & 부가 함수 정리

#### 📌 기본 수치 연산자

| 함수 / 연산 | 설명 | 예시 | 결과 |
|-------------|------|------|------|
| `a DIV b` | 정수 나눗셈 (소수점 버림, 몫) | `17 DIV 5` | `3` |
| `MOD(a, b)` | 나머지 연산 (a % b) | `MOD(17, 5)` | `2` |
| `a / b` | 일반 나눗셈 (소수점 포함) | `17 / 5` | `3.4` |

#### 🧮 수학 함수

| 함수 | 설명 | 예시 | 결과 |
|------|------|------|------|
| `FLOOR(x)` | 내림 (가장 가까운 아래 정수) | `FLOOR(3.9)` | `3` |
| `CEIL(x)` 또는 `CEILING(x)` | 올림 (가장 가까운 위 정수) | `CEIL(3.1)` | `4` |
| `ROUND(x, n)` | 반올림 (n자리까지) | `ROUND(3.1415, 2)` | `3.14` |
| `TRUNCATE(x, n)` | 소수점 n자리까지 자르고 버림 | `TRUNCATE(3.6789, 2)` | `3.67` |

#### 🧠 구간 분류 활용 예

| 목적 | 예시 | 설명 |
|------|------|------|
| 가격 구간 | `PRICE DIV 10000` | 만 원 단위 구간 (0: 1만원 미만, 1: 1만원대…) |
| 올림 구간 | `CEIL(PRICE / 30000)` | 3만원 단위로 구간 나누기 |
| 잔액 계산 | `MOD(PRICE, 10000)` | 10,000원 단위 나머지 계산 |

#### 🔤 부가 함수 (문자 처리 등)

| 함수 | 설명 | 예시 | 결과 |
|------|------|------|-------|
| `CONCAT(a, b)` | 문자열 연결 | `CONCAT('₩', PRICE)` | `'₩16000'` |
| `LPAD(str, len, pad)` | 왼쪽 채우기 | `LPAD('7', 3, '0')` | `'007'` |
| `RPAD(str, len, pad)` | 오른쪽 채우기 | `RPAD('A', 3, '.')` | `'A..'` |
| `DATE_FORMAT(date, format)` | 날짜 포맷 변환 | `DATE_FORMAT(NOW(), '%Y-%m')` | `'2025-06'` |